![](presentation/function_triangulation.png)

In [ ]:
from functools import partial, lru_cache

from ipywidgets import interact_manual
import numpy as np
import scipy.linalg
import scipy.spatial
import kwant

import adaptive
adaptive.notebook_extension()

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Define the lattice vectors of some common unit cells
hexegonal = ((0, 1, 0),
             (np.cos(-np.pi/6), np.sin(-np.pi/6), 0),
             (0, 0, 1))

simple_cubic = ((1, 0, 0),
                (0, 1, 0),
                (0, 0, 1))

fcc = ((0,.5,.5),
       (.5,.5,0),
       (.5,0,.5))

bcc = ((-.5, .5, .5),
       (.5,-.5, .5),
       (.5, .5,-.5))

lattices = dict(hexegonal=hexegonal, simple_cubic=simple_cubic, fcc=fcc, bcc=bcc)

In [ ]:
@lru_cache()
def create_syst(unit_cell):
    unit_cell_atoms = [( 0,  0,  0)]
    lat = kwant.lattice.Polyatomic(unit_cell, unit_cell_atoms, 'Some crystal')
   
    syst = kwant.Builder(kwant.TranslationalSymmetry(*lat.prim_vecs)) # infinite in all dimensions
    
    syst[lat.shape(lambda pos: True, (0, 0, 0))] = 6 # onsite
    syst[lat.neighbors()] = -1 # hopping

    syst = kwant.wraparound.wraparound(syst).finalized()

    return syst

def get_hull(unit_cell):
    syst = create_syst(unit_cell)
    A = get_A(syst)
    neighbours = kwant.linalg.lll.voronoi(A)
    lattice_points = np.concatenate(([[0,0,0]], neighbours))
    lattice_points = 2 * np.pi * (lattice_points @ A.T)
    vor = scipy.spatial.Voronoi(lattice_points)
    brillouin_zone = vor.vertices[vor.regions[vor.point_region[0]]]
    hull = scipy.spatial.ConvexHull(brillouin_zone)
    return hull

def energies(params, syst):
    H = syst.hamiltonian_submatrix(params=params)
    eigs = np.linalg.eigvalsh(H)
    return eigs

def momentum_to_lattice(k, syst):
    A = get_A(syst)
    k, residuals = scipy.linalg.lstsq(A, k)[:2]
    if np.any(abs(residuals) > 1e-7):
        raise RuntimeError("Requested momentum doesn't correspond"
                           " to any lattice momentum.")
    return k

def get_A(syst):
    B = np.asarray(syst._wrapped_symmetry.periods).T
    return np.linalg.pinv(B).T

def E(k, unit_cell):
    syst = create_syst(unit_cell)
    k_x, k_y, k_z = momentum_to_lattice(k, syst)
    return min(energies({'k_x': k_x, 'k_y': k_y, 'k_z': k_z}, syst))

In [ ]:
learners = []
for name, unit_cell in lattices.items():
    hull = get_hull(unit_cell)
    learner = adaptive.LearnerND(partial(E, unit_cell=unit_cell), hull)
    learner.fname = name
    learners.append(learner)
learner = adaptive.BalancingLearner(learners, strategy='npoints')
learner.load('lattices')

In [ ]:
# runner = adaptive.Runner(learner, goal=lambda l: l.learners[0].npoints > 5000) 
# runner.live_info()

In [ ]:
def select(name, learner=learner):
    return next(l for l in learner.learners if l.fname == name)

def iso(unit_cell, level=8.5):
    l = select(unit_cell)
    return l.plot_isosurface(level=level)

def plot_tri(unit_cell):
    from plot_tri import plot
    l = select(unit_cell)
    return plot(l)

In [ ]:
interact_manual(plot_tri, unit_cell=lattices.keys())

In [ ]:
interact_manual(iso, level=(-6, 9, 0.1), unit_cell=lattices.keys()) 

In [ ]:
import holoviews as hv
def plot_slice(unit_cell, xyz, value):
    learner = select(unit_cell)
    direction = {'x': 0, 'y': 1, 'z': 2}[xyz]
    return learner.plot_slice(cut_mapping={direction: value}, n=100)

hv.DynamicMap(plot_slice, kdims=['unit_cell', 'xyz', 'value']).redim.values(
    unit_cell=list(lattices.keys()), xyz=list('xyz'), value=np.linspace(-4, 4))